# 운영체제 과제2
201901758 임베디드 시스템공학과 유승태  
* 게임 과정
1. 초기상태
![초기 화면](./%EC%B4%88%EA%B8%B0%20%ED%99%94%EB%A9%B4.png)    
게임 초기화면에는 난이도를 방향키로 바꾼 후 엔터를 입력하면  
해당 난이도의 게임이 시작된다.  
난이도가 어려워 질 수록 상대 바의 속도가 증가한다.  
  

2. 게임 실행 중  
![게임 실행 중](./%EA%B2%8C%EC%9E%84%20%EC%8B%A4%ED%96%89%20%EC%A4%91.png)  
![실행중 2](./%EC%8B%A4%ED%96%89%EC%A4%91%202.png)    
게임이 실행되면 왼쪽 초록색바가 유저, 오른쪽 붉은색 바가 컴퓨터이다.  
상대방 벽쪽으로 공을 박아넣을 경우, stage가 상승하며 공의속도가 상승한다.  
현재 stage는 가운데 상단의 글을 통해 확인 가능하다.  

3. 게임 종료  
![game over 상태](./game%20over%20%EC%83%81%ED%83%9C.png)    
유저가 공을 받지 못할 경우, 게임이 종료되며, 몇stage까지 갔는지 표시된다.  
게임을 다시 진행할지 종료할지 방향키와 엔터로 선택이 가능하다.





In [ ]:
#include <Windows.h>
#include <stdlib.h>
#include <tchar.h>
#include <time.h>


#define WINDOW_WIDTH 800	// 창 너비
#define WINDOW_HEIGHT 600	// 창 높이



#define SELECT 0		// 난이도 선택 단계
#define PLAY 1			// 게임 진행 단계
#define GAMEOVER 2		// 게임 끝



// 난이도에 따른 enemy bar의 속도
#define EASY 3				//쉬움(속도)
#define MEDIUM 5			//보통(속도)
#define HARD 7				//어려움(속도)

// 바의 크기
#define BAR_HEIGHT 120
#define BAR_WIDTH 15

// 공의 반지름
#define RADIUS 15


int status = SELECT;	// 게임 시작 초기 상태
int difficulty = EASY;		// 난이도
int stage = 1;		// 스테이지
int restart = 1;	// 다시하기



// 공의 좌표와 스피드
int ball_cordinate[2] = { 380,280 };
int ball_offset[2] = { -9,9 };


// 적 바의 좌표와 스피드
int enemy_cordinate[2] = { 790,250 };
int enemy_offset;

// 유저의 바의 좌표와 스피드
int user_cordinate[2] = { 0,250 };
int user_offset = 20;


LRESULT CALLBACK WndProc(HWND hwnd, UINT iMsg, WPARAM wParam, LPARAM lParam);


int WINAPI WinMain(HINSTANCE hInstance, HINSTANCE hPrevInstance, LPSTR lpszCmdLine, int nCmdShow)
{
	HWND hwnd;
	MSG msg;
	WNDCLASS WndClass = { 0 };
	WndClass.style = CS_HREDRAW | CS_VREDRAW;						// 클래스 종류
	WndClass.lpfnWndProc = WndProc;									// 메시지 처리를 위한 함수 등록
	WndClass.hInstance = hInstance;									// 자신 모듈의 인스턴스 핸들
	WndClass.hIcon = LoadIcon(NULL, IDI_APPLICATION);				// 아이콘모양
	WndClass.hCursor = LoadCursor(NULL, IDC_ARROW);					// 커서모양
	WndClass.hbrBackground = CreateSolidBrush(RGB(255, 255, 255));	// 흰색 배경
	WndClass.lpszClassName = TEXT("Windows Class");					// 클래스이름

	RegisterClass(&WndClass);		// 윈도우 클래스 등록

	hwnd = CreateWindow(
		TEXT("Windows Class"),		// 클래스이름
		TEXT("하키게임"),			// 캡션 명
		WS_OVERLAPPEDWINDOW,		// 윈도우스타일
		10,							// x좌표
		10,							// y좌표
		WINDOW_WIDTH,				// 윈도우 가로크기
		WINDOW_HEIGHT,				// 윈도우 세로크기	CW_USEDEFAULT는 커널에 의해 기본값을 지정
		0,
		0,
		hInstance,					// 인스턴스 핸들
		0
	);

	SetTimer(hwnd, 1, 20, 0);	// 20ms로 타이머 설정. 20ms마다 업데이트
	ShowWindow(hwnd, nCmdShow);	// 윈도우 인스턴스 시각화



	while (GetMessage(&msg, 0, 0, 0))	// 메시지 루프에서 메시지 꺼냄
	{
		TranslateMessage(&msg);
		DispatchMessage(&msg);
	}

	return 0;
}



LRESULT CALLBACK WndProc(HWND hwnd, UINT iMsg, WPARAM wParam, LPARAM lParam)
{
	HDC hdc;					//디바이스콘텍스트
	PAINTSTRUCT ps;

	TCHAR buffer[5]; // stage를 담을 버퍼

	HBRUSH myBrush;
	HBRUSH oldBrush;

	srand((unsigned)time(NULL));


	switch (iMsg)
	{

		// 그리기
	case WM_PAINT:
		hdc = BeginPaint(hwnd, &ps);

		// 난이도 선택 단계
		if (status == SELECT) {
			TextOut(hdc, 270, 150, TEXT("201901758 유승태 운영체제 과제2"), 22);

			// 난이도 선택 텍스트
			TextOut(hdc, 350, 200, TEXT("난이도"), 3);
			TextOut(hdc, 360, 230, TEXT("쉬움"), 2);
			TextOut(hdc, 360, 260, TEXT("보통"), 2);
			TextOut(hdc, 350, 290, TEXT("어려움"), 3);


			if (difficulty == EASY) {
				TextOut(hdc, 320, 230, TEXT("▶"), 1);
			}
			else if (difficulty == MEDIUM) {
				TextOut(hdc, 320, 260, TEXT("▶"), 1);
			}
			else if (difficulty == HARD) {
				TextOut(hdc, 320, 290, TEXT("▶"), 1);
			}

		}

		// 게임 플레이 단계
		else if (status == PLAY) {

			// stage를 텍스트로 그림.
			TextOut(hdc, 330, 10, TEXT("STAGE"), 5);
			wsprintf(buffer, TEXT("%d"), stage);
			TextOut(hdc, 380, 10, buffer, strlen(buffer));

			// 공이 튀기는 장소를 구분하는 선
			Rectangle(hdc, 0, 50, 800, 60);

			// user, 초록색
			myBrush = CreateSolidBrush(RGB(0, 255, 0));
			oldBrush = (HBRUSH)SelectObject(hdc, myBrush);
			Rectangle(hdc, user_cordinate[0], user_cordinate[1], user_cordinate[0] + BAR_WIDTH, user_cordinate[1] + BAR_HEIGHT);
			SelectObject(hdc, oldBrush);
			DeleteObject(myBrush);

			// enemy, 빨간색
			myBrush = CreateSolidBrush(RGB(255, 0, 0));
			oldBrush = (HBRUSH)SelectObject(hdc, myBrush);
			Rectangle(hdc, enemy_cordinate[0] - 21, enemy_cordinate[1], enemy_cordinate[0] + BAR_WIDTH - 21, enemy_cordinate[1] + BAR_HEIGHT);
			SelectObject(hdc, oldBrush);
			DeleteObject(myBrush);

			// ball, 검정색
			myBrush = CreateSolidBrush(RGB(0, 0, 0));
			oldBrush = (HBRUSH)SelectObject(hdc, myBrush);
			Ellipse(hdc, ball_cordinate[0], ball_cordinate[1], ball_cordinate[0] + RADIUS, ball_cordinate[1] + RADIUS);
			SelectObject(hdc, oldBrush);
			DeleteObject(myBrush);


		}

		// 게임 종료 단계
		else if (status == GAMEOVER) {
			TextOut(hdc, 330, 100, TEXT("달성한 STAGE: "), 11);
			wsprintf(buffer, TEXT("%d"), stage);
			TextOut(hdc, 440, 100, buffer, strlen(buffer));

			TextOut(hdc, 360, 230, TEXT("다시하기"), 4);
			TextOut(hdc, 360, 260, TEXT("종료"), 2);

			if (restart == 1) {
				TextOut(hdc, 320, 230, TEXT("▶"), 1);
			}
			else if (restart == 0) {
				TextOut(hdc, 320, 260, TEXT("▶"), 1);
			}

		}
		EndPaint(hwnd, &ps);

		break;

		// 키보드 입력
	case WM_KEYDOWN:

		// 난이도 선택 단계
		if (status == SELECT) {
			switch (wParam) {

				// 엔터를 누를 경우 해당 난이도로 게임 시작
			case VK_RETURN:
				status = PLAY;
				enemy_offset = difficulty;
				break;

				// 위 아래 방향키를 통해 난이도 선택
			case VK_UP:
				if (difficulty == MEDIUM) {
					difficulty = EASY;
				}
				else if (difficulty == HARD) {
					difficulty = MEDIUM;
				}
				break;

			case VK_DOWN:
				if (difficulty == EASY) {
					difficulty = MEDIUM;
				}
				else if (difficulty == MEDIUM) {
					difficulty = HARD;
				}
				break;
			default:
				break;
			}
		}

		// 게임 플레이 단계
		else if (status == PLAY) {
			switch (wParam) {

				// 화살표 입력에 따른 user bar 위치 업데이트
			case VK_DOWN:
				user_cordinate[1] += user_offset;
				if (user_cordinate[1] + BAR_HEIGHT >= 560) {
					user_cordinate[1] = 440;
				}
				break;
			case VK_UP:
				user_cordinate[1] -= user_offset;
				if (user_cordinate[1] <= 60) {
					user_cordinate[1] = 60;
				}
				break;
			}
		}
		// 게임 종료 단계
		else if (status == GAMEOVER) {
			switch (wParam) {
				// 엔터 입력시, 상태에 따라 다시하기 or 종료
			case VK_RETURN:
				// 다시하기, 상태 초기화.
				if (restart == 1) {
					status = SELECT;
					// 공의 좌표와 스피드
					ball_cordinate[0] = 380;
					ball_cordinate[1] = 280;
					ball_offset[0] = -9;
					ball_offset[1] = 9;
					enemy_cordinate[0] = 790;
					enemy_cordinate[1] = 250;
					user_cordinate[0] = 0;
					user_cordinate[1] = 250;
					stage = 1;
				}
				// 종료.
				else if (restart == 0) {
					PostQuitMessage(0);
				}
				break;

				// 위 아래 방향키를 통해 다시하기 선택
			case VK_UP:
				if (restart == 0) {
					restart = 1;
				}
				break;

			case VK_DOWN:
				if (restart == 1) {
					restart = 0;
				}
				break;
			default:
				break;
			}
		}
		break;

		// 타이머
	case WM_TIMER:
		if (status == PLAY) {
			// 공의 위치 업데이트
			ball_cordinate[0] += ball_offset[0];
			ball_cordinate[1] += ball_offset[1];

			// 공이 위 아래로 튕길 경우.
			if (ball_offset[1] < 0 && ball_cordinate[1] < 60) {
				ball_offset[1] = -ball_offset[1];
			}
			if (ball_offset[1] > 0 && ball_cordinate[1] > 540) {
				ball_offset[1] = -ball_offset[1];
			}

			// 공이 user bar에 튕길 경우
			if (ball_offset[0] < 0) {
				if (ball_cordinate[0]<user_cordinate[0] + BAR_WIDTH && ball_cordinate[1] + RADIUS> user_cordinate[1] && ball_cordinate[1] + RADIUS < user_cordinate[1] + BAR_HEIGHT) {
					ball_offset[0] = -ball_offset[0];
				}
			}
			// 공이 enemy bar에 튕길 경우
			if (ball_offset[0] > 0) {
				if (ball_cordinate[0] + 2 * RADIUS > enemy_cordinate[0] && ball_cordinate[1] + RADIUS > enemy_cordinate[1] && ball_cordinate[1] + RADIUS < enemy_cordinate[1] + BAR_HEIGHT) {
					ball_offset[0] = -ball_offset[0];
				}
			}

			// 공이 user방향 벽으로 나갈 경우
			if (ball_cordinate[0] + 2 * RADIUS < 0) {
				status = GAMEOVER;
			}


			// 공이 enemy방향 벽으로 나갈 경우
			// stage 한단계 +
			// 공의 스피트 2만큼 증가.
			if (ball_cordinate[0] + 2 * RADIUS > 800) {
				stage++;
				ball_cordinate[0] = 380;
				ball_cordinate[1] = 280;
				ball_offset[0] = -(ball_offset[0] + 2);
				if (ball_offset[1] > 0) {
					ball_offset[1] = rand() % 10 + 6;
				}
				else {
					ball_offset[1] = -(rand() % 10 + 6);
				}

			}



			// enemy bar의 위치 업데이트
			// bar가 공의 위치를 따라가는 방식으로 업데이트
			if (enemy_cordinate[1] + BAR_HEIGHT / 2 > ball_cordinate[1] + RADIUS && enemy_offset > 0) {
				enemy_offset = -enemy_offset;
			}
			else if (enemy_cordinate[1] + BAR_HEIGHT / 2 < ball_cordinate[1] + RADIUS && enemy_offset < 0) {
				enemy_offset = -enemy_offset;
			}
			enemy_cordinate[1] += enemy_offset;

			if (enemy_cordinate[1] + BAR_HEIGHT >= 560) {
				enemy_cordinate[1] = 440;
			}
			if (enemy_cordinate[1] <= 60) {
				enemy_cordinate[1] = 60;
			}


		}

		InvalidateRect(hwnd, NULL, TRUE);
		break;	

		// 창을 닫으면 종료
	case WM_DESTROY:
		PostQuitMessage(0);
		break;
	default:
		break;
	}

	return DefWindowProc(hwnd, iMsg, wParam, lParam);
}

